##### Blob 스토리지에 업로드한 PDF 파일을 AI Search의 인덱스가 텍스트 기반으로 저장(현재 임베딩 모델 사용하지 않아, 벡터화 하지 않음)하고,
##### 사용자 질의 시, 데이터를 추출하여 LLM이 자연어로 생성 및 응답

In [2]:

import os
import base64
from openai import AzureOpenAI
from dotenv import load_dotenv

# .env 환경변수 로드
load_dotenv()
endpoint = os.getenv("ENDPOINT_URL", "https://8a000-openai.openai.azure.com/")                          # OpenAI - EndPoint
deployment = os.getenv("AZURE_OAI_DEPLOYMENT", "gpt-4o-mini")                                           # OpenAI - Model Name
subscription_key = os.getenv("AZURE_CHRIS_KEY", "REPLACE_WITH_YOUR_KEY_VALUE_HERE")                     # OpenAI - Key

search_endpoint = os.getenv("SEARCH_ENDPOINT", "https://8ai022-ai-search.search.windows.net")           # AI Search - EndPoint
search_key = os.getenv("AZURE_SEARCH_KEY", "put your Azure AI Search admin key here")                   # AI Search - Key
search_index = os.getenv("SEARCH_INDEX_NAME", "msai-ot-index")                                          # AI Search - Index Name

# Initialize Azure OpenAI client with key-based authentication
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=subscription_key,
    api_version="2025-01-01-preview",
)

# IMAGE_PATH = "YOUR_IMAGE_PATH"
# encoded_image = base64.b64encode(open(IMAGE_PATH, 'rb').read()).decode('ascii')

# Prepare the chat prompt
chat_prompt = [
    {
        "role": "system",
        "content": "너는 Microsoft ai school 오리엔테이션 강사야."
    },  
    {
        "role": "user",
        "content": "MS AI School에 대해서 설명해줘."
    }
]

# Include speech result if speech is enabled
messages = chat_prompt

# Generate the completion
completion = client.chat.completions.create(
    model=deployment,
    messages=messages,
    max_tokens=6553,
    temperature=0.7,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None,
    stream=False,
    extra_body={
      "data_sources": [{
          "type": "azure_search",
          "parameters": {
            "endpoint": f"{search_endpoint}",
            "index_name": f"{search_index}",            # Index Name
            "semantic_configuration": "default",
            "query_type": "semantic",                   # 의미체계(semantic)로 검색. 키워드는 simple
            "fields_mapping": {},
            "in_scope": True,
            #"role_information": "너는 margie 여행사의 여행 전문가야. 사용자가 업로드한 Azure blob storage의 데이터들을 기반으로만 답변해.",
            "filter": None,
            "strictness": 3,
            "top_n_documents": 5,
            "authentication": {
              "type": "api_key",
              "key": f"{search_key}"
            }
          }
        }]
    }
)

print(completion.choices[0].message.content)
#print(completion.choices[0].message.content)
    

MS AI School은 Microsoft에서 제공하는 AI 관련 교육 프로그램입니다. 이 프로그램은 AI 기술의 기초부터 고급 내용까지 폭넓은 주제를 다루며, 다양한 학습 자료와 강의를 제공합니다 .

AI School의 주요 내용은 다음과 같습니다:

- **교육 과정**: AI School은 Azure Machine Learning, Azure OpenAI Studio 등 다양한 AI 도구와 플랫폼에 대한 교육을 포함합니다. 이 과정에서는 실제 사례를 통한 실습과 이론 교육이 결합되어 있어 학습자가 기술을 효과적으로 이해하고 활용할 수 있도록 돕습니다 [doc3][doc4].

- **학습 자료**: Microsoft Learn과 같은 플랫폼을 통해 AI 관련 자료와 리소스를 제공하며, 학습자는 자율적으로 학습할 수 있는 기회를 가집니다 .

- **AI 기술 적용**: AI School은 AI 기술이 실제 비즈니스 및 산업에 어떻게 적용될 수 있는지를 강조하며, 이를 통해 학습자는 실제 상황에서 AI를 활용할 수 있는 능력을 기를 수 있습니다 [doc4].

이 프로그램은 AI에 대한 깊이 있는 지식과 실습 경험을 제공하여, 참가자들이 AI 기술을 효과적으로 활용할 수 있도록 지원합니다.


In [40]:
citations = completion.choices[0].message.context['citations']
for i in range(len(citations)) :
    print(f'Title : {citations[i]["title"]}, URL : {citations[i]["url"]}')
    print('=' * 50)


Title : (Microsoft Academy) Orientation_AI.pdf, URL : https://8ai022storage.blob.core.windows.net/fileupload-msai-ot-index/(Microsoft%20Academy)%20Orientation_AI.pdf
Title : (Microsoft Academy) Orientation_AI.pdf, URL : https://8ai022storage.blob.core.windows.net/fileupload-msai-ot-index/(Microsoft%20Academy)%20Orientation_AI.pdf
Title : (Microsoft Academy) Orientation_AI.pdf, URL : https://8ai022storage.blob.core.windows.net/fileupload-msai-ot-index/(Microsoft%20Academy)%20Orientation_AI.pdf
Title : (Microsoft Academy) Orientation_AI.pdf, URL : https://8ai022storage.blob.core.windows.net/fileupload-msai-ot-index/(Microsoft%20Academy)%20Orientation_AI.pdf
Title : (Microsoft Academy) Orientation_AI.pdf, URL : https://8ai022storage.blob.core.windows.net/fileupload-msai-ot-index/(Microsoft%20Academy)%20Orientation_AI.pdf
